In [ ]:
'''
this notebook builds a CNN model for audio classification which is using
A. HDF5 file
B. Custom generator to extract images from HDF5 file and feed into Neural Network
C. Need TensorFlow 2.1.0
D. It is based on transfer learnng (TL) which is using MobileNet CNN model

'''

In [ ]:
!pip install tensorflow==2.1.0

In [ ]:
import tensorflow as tf

import tensorflow_hub as hub
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.metrics import classification_report

In [ ]:
def image_feature_generator(hdf5_path, bs, startIndex, endIndex, numClasses=2, mode="train"):
    dataset = h5py.File(hdf5_path, "r")
    train_labels=np.array(dataset["train_labels"])
    i=startIndex
    while True:
        train_batch_imgs = []
        labels = []
        
        while len(train_batch_imgs) < bs:
            if endIndex < i:
                break;
            
            img=(dataset['train_img'])[i]
            img=img/255.
            train_batch_imgs.append(img) 
            labels.append(train_labels[i])
            
            i = i+1

        yield (np.array(train_batch_imgs), np.array(labels))

In [ ]:
hdf5_path = 'Audio_Classification_Train_HDF5_0423_v1_.hdf5'

dataset = h5py.File(hdf5_path, "r")
train_labels=np.array(dataset["train_labels"])
train_labels = train_labels.reshape((len(train_labels),-1))

In [ ]:
img=(dataset['train_img'])[0]
img=img/255.
img.shape

In [ ]:
trainGen = image_feature_generator(hdf5_path, 32,0, 10000, 2, mode="train")
valGen = image_feature_generator(hdf5_path, 32,10001, 11191, 2, mode="train")

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(224 * 224 * 3,), activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(2, activation="softmax"))

opt = SGD(lr=1e-3, momentum=0.9, decay=1e-3 / 25)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [ ]:
model = tf.keras.Sequential([
                    hub.KerasLayer(
                        "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                        output_shape=[1280], trainable=False
                    ),
                    tf.keras.layers.Dropout(0.4),
                    tf.keras.layers.Dense(2, activation='softmax')
                    ])

model.build([None, 224, 224, 3])
model.summary()

model.compile( optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
steps_per_epoch = np.ceil(10000/32)
val_steps_per_epoch = np.ceil(1191/32)

hist = model.fit_generator( trainGen, epochs=10, verbose=1,
                            steps_per_epoch=steps_per_epoch,
                            validation_data=trainGen,
                            validation_steps=val_steps_per_epoch
                           ).history